# LLM-ZOOMCAMP HW-1

## Q1
Based on the output of 'curl http://localhost:9200',
version.build_hash reads:
42f05b9372a9a4a470db3b52817899b99a76ee73

## Q2

We use 'index' to add data to ES. See below:

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)        

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions-hw'
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw'})

In [4]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 239.73it/s]


## Q3
Top hit has a score of 44.50. See below:

In [5]:
query = 'How do execute a command on a Kubernetes pod?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [6]:
response = es_client.search(index=index_name, body=search_query)
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [7]:
response = es_client.search(index=index_name, body=search_query)
result_scores = []
for hit in response['hits']['hits']:
    result_scores.append(hit['_score'])
result_scores

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

## Q4
The question of the 3rd source is: 'How do I copy files from a different folder into docker container’s working directory?'. See below:

In [26]:
query = 'How do copy a file to a Docker container?'
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # , "section"
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [27]:
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [28]:
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]['_source']['text']

'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan'

## Q5
length of the resulting prompt is 1446. See below:

In [29]:
context = ''
context_template = """
Q: {question}
A: {text}
""".strip()
for hit in response['hits']['hits']:
    question = hit['_source']['question']
    text = hit['_source']['text']
    new_context = context_template.format(question=question, text=text)
    context = context + f"{new_context}\n\n"
context.strip()
print(context)


Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [30]:
query = 'How do copy a file to a Docker container?'
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
"""
prompt = prompt_template.format(question=query, context=context).strip()
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [31]:
len(prompt)

1446

## Q6
our prompt has 320 tokens. See below:

In [32]:
import tiktoken

In [33]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [34]:
encoded = encoding.encode(prompt)

In [35]:
len(encoded)

320